<p style="float: left;"><a href="merge-sort.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="for-comprehensions.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Definition of class List II: Higher-Order Methods

**Functional programming** languages **enable programmers to write general functions
which implement patterns by means of higher order functions**. We now
discuss a set of commonly used higher-order functions, which are implemented as
methods in class `List`.

We can identify several patterns of computation over lists, like:

* Transforming every element of a list in some way.
* Extracting from a list all elements satisfying a criterion.
* Combine the elements of a list using some operator.

## Mapping

A common operation is to transform each element of a list
and then return the lists of results.

```scala
def map[B](f: A => B): List[B] = this match {
    case Nil     => this
    case x :: xs => f(x) :: xs.map(f)
}
```


For instance, to scale each element of a list by a
given factor:

In [1]:
import scala.util.Random

val rand    = new Random
val numbers =  for (x <- 1 to 3) yield rand.nextDouble()
val randInt = rand.nextInt()
val scale   = (if (randInt >= 0) randInt else -1 * randInt) % 5 + 1

val scaledNumbers = numbers map (_ * 2)

import scala.util.Random


rand: Random = scala.util.Random@4690d39e
numbers: IndexedSeq[Double] = Vector(
  0.8977277360595421,
  0.7475959172344271,
  0.7657906659460046
)
randInt: Int = -717768136
scale: Int = 2
scaledNumbers: IndexedSeq[Double] = Vector(
  1.7954554721190843,
  1.4951918344688542,
  1.5315813318920093
)

As another example, consider the problem of returning a given column of a matrix
which is represented as a list of rows, where each row is again a list.

In [2]:
import scala.util.Random

def column[A](xs: List[List[A]], indexCol: Int): List[A] =
    xs map (row => row(indexCol))

val (rows, columns) = (2, 3)

val grid = (
    for (x <- 1 to rows) 
        yield (for (x <- 1 to columns) yield rand.nextDouble()).toList
).toList

val indexCol = columns - 1

column(grid, indexCol)

import scala.util.Random


defined function column
rows: Int = 2
columns: Int = 3
grid: List[List[Double]] = List(
  List(0.7594093769461904, 0.027196966877725992, 0.47694865449501),
  List(0.7284617583918841, 0.6469679928030594, 0.8517658821658202)
)
indexCol: Int = 2
res2_5: List[Double] = List(0.47694865449501, 0.8517658821658202)

## Foreach

Closely related to `map` is the `foreach` method, which applies a given function to all
elements of a list, but does not construct a list of results. The function is thus applied
only for its side effect.

```scala
def foreach(f: A => Unit): Unit =
    this match {
        case Nil     => ()
        case x :: xs => f(x)
                        xs.foreach(f)
```


In [3]:
(1 to 3).toList foreach println // 1, 2, 3

1
2
3


## Filtering

Another common operation selects from a list all elements fulfilling a given criterion. The function criteria is also known as `predicate`.

```scala
def filter(p: A => Boolean): List[A] = this match {
    case Nil     => this
    case x :: xs => if (p(x)) x :: xs.filter(p) else xs.filter(p)
}
```

For instance, to return a list of all positive elements in some
given lists of integers:

In [ ]:
import scala.util.Random

val rand = new Random
val randNumbers = for (_ <- 1 to 5) yield rand.nextInt()
val positiveNumbers = randNumbers filter (x => x > 0)

## Folding

Another common operation is to combine the elements of a list with some operator. For instance:

* `sum(List(x1, ..., xn))`  $ = 0 + x_1 + \dots + x_n$
* `product(List(x1, ..., xn))` $ = 1 * x_1 * \dots * x_n$

### Fold left

```scala
def foldLeft[B](z: B)(op: (B, A) => B): B = this match {
    case Nil     => z                            // `foldLeft [1]`
    case x :: xs => (xs foldLeft op(z, x))(op)   // `foldLeft [2]`
}
```

Applies a binary operator $\Theta$ to a start value $z$ and all elements of a list $x_1, x_2, .., x_n$, going left to right.

**Example:**

```scala
List(1, 2, 3).foldLeft(0)(_ + _)

```

Step-by-step evaluation:


```scala
>> List(1, 2, 3).foldLeft (0) (_ + _)        
>> List(2, 3).foldLeft (1) (_ + _)     // `foldLeft [2]`
>> List(3).foldLeft (3) (_ + _)        // `foldLeft [2]`
>> List().foldLeft (6) (_ + _)         // `foldLeft [2]`
>> 6                                   // `foldLeft [1]`
```

**`foldLeft` generates the <span style="color:red">left associative expression</span> $ (\cdots ((z \ \Theta \ x_1) \ \Theta \ x_2) \ \cdots ) \ \Theta \ x_n$ which can be simplified by:**

$$
z \ \Theta \ x_1 \ \Theta \ x_2 \ \cdots \ \Theta \ x_n
$$

### Fold right


```scala
def foldRight[B](z: B)(op: (A, B) => B): B = this match {
    case Nil     => z                                // `foldRight [1]`
    case x :: xs => op(x, (xs foldRight z)(op))      // `foldRight [2]`
}
```

Applies a binary operator $\Theta$ for all elements of a list $x_1, x_2, .., x_n$ and a start value $z$, going right to left.

**Example:**

```scala
List(1, 2, 3).foldRight(0)(_ + _)

```

Step-by-step evaluation:

```scala
>> List(1, 2, 3).foldRight (0) (_ + _)        
>> 1 + List(2, 3).foldRight 0 (_ + _)            // `foldRight [2]`
>> 1 + (2 + List(3).foldRight 0) (_ + _))        // `foldRight [2]`
>> 1 + (2 + (3 + List().foldRight 0) (_ + _)))   // `foldRight [2]`
>> 1 + (2 + (3 + 0))                             // `foldRight [1]`
>> 1 + (2 + 3)                                   // stack call
>> 1 + 5                                         // stack call
>> 6                                             // stack call
```

**So, in general, `foldRight` produces a <span style="color:red">right-associative expression:</span>**

$$x_1 \ \Theta \ (x_2 \ \Theta \ \cdots \ (x_n \ \Theta \ z) \cdots )$$

### FoldLeft vs. FoldRight

Both `foldLeft` and `foldRight` are very similar, but the key difference is the way the expressions are generated.

- `foldLeft` is _tail-recursive_ **(it evaluates the accumulator at each step)**
- `foldRight` is non _tail-cursive_ **(large lists can cause overflow)**
- Use `foldRight` when you need to preserve the structure _right-to-left_ or you are working with lazzy data stractures such as `Strem` or `LazyList` because you can _short-circuit_ without generating the whole structure.


#### Structure

Variance of the structure using `foldLeft` and `foldRight`.

In [15]:
import scala.language.postfixOps

val yys = List (
    List.range(1, 5),
    List.range(6, 10),
    List.range(11, 15)
)

import scala.language.postfixOps


yys: List[List[Int]] = List(
  List(1, 2, 3, 4),
  List(6, 7, 8, 9),
  List(11, 12, 13, 14)
)

Flatten list from _left-to-right_.

In [ ]:
(yys foldLeft (Nil)) ((xxs, xs) => xxs ::: xs) // List(1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14)

Flatten list from _right-to-left_.

In [ ]:
(yys foldRight (Nil)) ((xs, xxs) => xxs ::: xs) // List(11, 12, 13, 14, 6, 7, 8, 9, 1, 2, 3, 4)

#### Short-circuit

- The method `foldRight` is only relevant if your operator itself **doesn’t force the second argument**, like in boolean `&&` or `||` and you work with lazzy data structures.
- On the other hand `foldLeft` can never _short-circuit_, you process the whole structure.

**Example `foldRight`:**


```scala
def foldRight[B](z: B)(op: (A, B) => B): B = this match {
    case Nil     => z                                // `foldRight [1]`
    case x :: xs => op(x, (xs foldRight z)(op))      // `foldRight [2]`
}

```

```scala
(False :: True :: Nil).foldRight (True) (_ && _)

```

Step-by-step evaluation:

```scala
>> (False :: True :: Nil).foldRight (True) (_ && _)        
>> False && ((True :: Nil).foldRight (True) (_ + _))     // `foldRight [2]`
>> False                                                 // Short-cutting

```


**Example `foldRight`:**

```scala
def foldLeft[B](z: B)(op: (B, A) => B): B = this match {
    case Nil     => z                            // `foldLeft [1]`
    case x :: xs => (xs foldLeft op(z, x))(op)   // `foldLeft [2]`
}

```

```scala
(False :: True :: Nil).foldLeft (True) (_ && _)

```

Step-by-step evaluation:

```scala
>> (False :: True :: Nil).foldLeft (True) (_ && _)     
>> (True :: Nil).foldLeft (False) (_ && _)       // `foldLeft [2]`
>> (Nil).foldLeft (False) (_ && _)               // `foldLeft [2]`
>> False                                         // `foldLeft [1]`

```

The method `reduceLeft` and `reduceRight` behaves similar to they analogous methods `foldLeft` and `foldRight`.
<span style="color:red">**But they require that the list is not empty.**</span>

### ReduceLeft & ReduceRight

```scala
def reduceLeft(op: (A, A) => A): A = this match {
    case Nil     => error("Nil.reduceLeft")    
    case x :: xs => (xs foldLeft x)(op)        
}

def reduceRight(op: (A, A) => A): A = this match {
    case Nil      => error("Nil.reduceRight")        
    case x :: Nil => x                              
    case x :: xs  => op(x, xs reduceRight op)        
}
```

## Nested Mappings

We can employ higher-order list processing functions to express many computations that are normally expressed as nested loops in imperative
languages.

As an example, consider the following problem: Given a positive integer $n$, find all
pairs of positive integers $i$ and $j$, where $1 \leq j \leq i < n$ such that $i + j$ is prime.

For instance, if $n = 7$ the pairs are:

| i | j | i + j |
|---|---|-------|
| 2 | 1 | 3     |
| 3 | 2 | 5     |
| 4 | 1 | 5     |
| 4 | 3 | 7     |
| 5 | 2 | 7     |
| 6 | 1 | 7     |
| 6 | 5 | 11    |



In a first step, one generates the sequence of all pairs $(i, j)$ of integers such that $1 \leq j < i < n$.

In [6]:
val n = 7

List.range(1, n)
    .map(i => List.range(1, i).map(j => (i, j)))

n: Int = 7
res6_1: List[List[(Int, Int)]] = List(
  List(),
  List((2, 1)),
  List((3, 1), (3, 2)),
  List((4, 1), (4, 2), (4, 3)),
  List((5, 1), (5, 2), (5, 3), (5, 4)),
  List((6, 1), (6, 2), (6, 3), (6, 4), (6, 5))
)

Secondly, combine all sublists (flatting) using foldRight with the concat operator `:::`,

In [7]:
List.range(1, n)
    .map(i => List.range(1, i).map(j => (i, j)))
    .foldRight(List[(Int, Int)]()){(x, xs) => x ::: xs}

res7: List[(Int, Int)] = List(
  (2, 1),
  (3, 1),
  (3, 2),
  (4, 1),
  (4, 2),
  (4, 3),
  (5, 1),
  (5, 2),
  (5, 3),
  (5, 4),
  (6, 1),
  (6, 2),
  (6, 3),
  (6, 4),
  (6, 5)
)

finally, we filter the pairs that their sum is prime.

In [8]:
def isPrime(n: Int): Boolean = {
    def nestedIsPrime(n: Int, i: Int): Boolean = { 
        if (n <= 2) n == 2
        else if (n % i == 0) false
        else if (n < i * i) true
        else nestedIsPrime(n, i + 1)
    }
    nestedIsPrime(n, 2)
}

val n = 7

List.range(1, n)
    .map(i => List.range(1, i).map(j => (i, j)))
    .foldRight(List[(Int, Int)]()){(x, xs) => x ::: xs}
    .filter(pair => isPrime(pair._1 + pair._2))

defined function isPrime
n: Int = 7
res8_2: List[(Int, Int)] = List(
  (2, 1),
  (3, 2),
  (4, 1),
  (4, 3),
  (5, 2),
  (6, 1),
  (6, 5)
)

## Flattening Maps

The combination of mapping and then concatenating sublists
resulting from the map is so common that we there is a special method for it in class.

```scala
def flatMap[B](f: A => List[B]): List[B] = this match {
    case Nil     => Nil
    case x :: xs => f(x) ::: (xs flatMap f)
}
```

With `flatMap`, the pairs-whose-sum-is-prime expression could have been written
more concisely as follows.

In [9]:
def isPrime(n: Int): Boolean = {
    def nestedIsPrime(n: Int, i: Int): Boolean = { 
        if (n <= 2) n == 2
        else if (n % i == 0) false
        else if (n < i * i) true
        else nestedIsPrime(n, i + 1)
    }
    nestedIsPrime(n, 2)
}

val n = 7

List.range(1, n)
    .flatMap(i => List.range(1, i).map(j => (i, j)))
    .filter(pair => isPrime(pair._1 + pair._2))

defined function isPrime
n: Int = 7
res9_2: List[(Int, Int)] = List(
  (2, 1),
  (3, 2),
  (4, 1),
  (4, 3),
  (5, 2),
  (6, 1),
  (6, 5)
)

<p style="float: left;"><a href="merge-sort.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="for-comprehensions.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>